In [124]:
function display_position(pos_matrix, num_lanes, time, traffic_light)
    street = []
    for _ in 1:num_lanes
        lane = collect("=" ^ traffic_light)
        lane[traffic_light] = '|'
        push!(street, lane)
    end
    
    for c in 1:num_cars
        lane = street[cars[c][2]]
        lane[pos_matrix[c, time]] = Char('0' + c)
    end
    
    for l in street
        println(join(l), "\n")
    end
end

display_position (generic function with 2 methods)

In [125]:
num_cars = 6
car = [-1, 1, 1, 1]                                                 # start lead car at pos = -1
cars = [car]
for _ in 1:num_cars-1
    car = []
    speed = rand(1:3)                                               # randomize speed    
    spacing = rand(1:6)                                             # randomize spacing
    lane = 1                                                        # all cars start in lane 1
    pos = cars[end][1] - 1 - spacing - rand(1:6)                    # next car trails the previous car by (spacing + random amount)
    car = [pos, lane,spacing, speed]
    push!(cars, car)
end

pos_of_last_car = cars[end][1]                                      # get pos of last car
shift_pos = 1 - pos_of_last_car

for car in cars
    car[1] += shift_pos                                             # shift pos, last car starts at pos = 1, negative numbers are awkward
end

time = 130                                                          # num time steps to be ran
pos_matrix = zeros(Int, num_cars, time)                             # record pos of each car over time
for c in 1:num_cars
    pos_matrix[c, 1] = cars[c][1]                                   # t = 1
end

traffic_light = 110                                                 # cars will stop at the traffic light
num_lanes = 3
println("t = 1")
display_position(pos_matrix, num_lanes, 1, traffic_light)
println()

for t in 2:time
    println("t = ", t)
    for car_id in 1:length(cars)
        car = cars[car_id]
        speed = car[4]
        
        if car[1] + speed > traffic_light                           # if car is at a traffic light car does not move
            for n in 1:speed
                if car[1] + (speed-n) > traffic_light
                    continue
                else
                    car[1] += speed - n
                    break
                end
            end
            pos_matrix[car_id, t] = car[1]
            continue
        end
        
        
        move = true
        
        for j in 1:length(cars)
            other_car = cars[j]
            dist = other_car[1] - car[1]
            if dist > 0                                            # check if car is in front
                if (dist - speed) <= car[3]                        # if car is closer than (spacing + speed) car does not move
                    if other_car[2] == car[2]                      # check if car is in same lane
                        
                        switch_down = true
                        if car[2] <= 1
                            switch_down = false
                        end
                        for c in cars
                        #   if other_car is in next lane AND is within + OR - 1 of curr_car pos: switch_lane = false
                            if c[2] == car[2]-1 && (c[1] == car[1] || c[1] == car[1]+1 || c[1] == car[1]+1)
                                switch_down = false
                            end
                        end
                        
                        switch_up = true
                        if car[2] >= num_lanes
                            switch_up = false
                        end
                        for c in cars
                        #   if other_car is in next lane AND is within + OR - 1 of curr_car pos: switch_lane = false
                            if c[2] == car[2]+1 && (c[1] == car[1] || c[1] == car[1]+1 || c[1] == car[1]+1)
                                switch_up = false
                            end
                        end
                        
                        if !switch_up && !switch_down
                            move = false                          # damn they got me boxed in >:{
                        elseif switch_up
                            car[2] += 1                           # switch up one lane
                        else
                            car[2] -= 1                           # switch down one lane
                        end
                        
                    end
                end
            end
        end
        
        if move
            car[1] += speed                                        # car moves per speed
        end
            
        pos_matrix[car_id,t] = car[1]                              # save pos for car c at time t
    end
    
    display_position(pos_matrix, num_lanes, t, traffic_light)
    println()
end

t = 1
6=======5========4==========3=====2=======1==================================================================|

=============================================================================================================|

=============================================================================================================|


t = 2
==6======5=========4==========3====2=======1=================================================================|

=============================================================================================================|

=============================================================================================================|


t = 3
====6=====5==========4==============2=======1================================================================|

================================3============================================================================|

====================================================================================